# Testing file 
### where we evaluate BEUTEL's models using the test set

## Preliminaries

In [1]:
import pandas as pd
from tensorflow.data import Dataset
from tensorflow.keras.optimizers import Adam, Adagrad


from util.load_data import load_data
from util.evaluation import *
from models.beutel.models import *
from models.beutel.learning import train_loop as beutel_train

In [2]:
batch_size = 32
epochs = 10
learning_rate = 0.01
opt = Adam(learning_rate=learning_rate)

In [3]:

FAIR_COEFFS = [1.]

In [4]:
test_loop = 1

## Load data

In [5]:
data_name = 'adult'

In [6]:
x_train, y_train, a_train = load_data(data_name, 'train')
raw_data = (x_train, y_train, a_train)

In [7]:
xdim = x_train.shape[1]
ydim = y_train.shape[1]
adim = a_train.shape[1]
zdim = 8

In [8]:
train_data = Dataset.from_tensor_slices((x_train, y_train, a_train))
train_data = train_data.batch(batch_size, drop_remainder=True)
train_data

2022-05-25 19:27:59.413872: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-05-25 19:27:59.472832: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-25 19:27:59.473361: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1650 computeCapability: 7.5
coreClock: 1.56GHz coreCount: 16 deviceMemorySize: 3.82GiB deviceMemoryBandwidth: 119.24GiB/s
2022-05-25 19:27:59.473648: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-05-25 19:27:59.475961: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-05-25 19:27:59.478036: I tensorflow/stream_executor/plat

<BatchDataset shapes: ((32, 112), (32, 1), (32, 1)), types: (tf.float64, tf.float64, tf.float64)>

In [9]:
x_valid, y_valid, a_valid = load_data(data_name, 'valid')

valid_data = Dataset.from_tensor_slices((x_valid, y_valid, a_valid))
valid_data = valid_data.batch(batch_size, drop_remainder=True)

In [10]:
x_test, y_test, a_test = load_data(data_name, 'test')

test_data = Dataset.from_tensor_slices((x_test, y_test, a_test))
test_data = test_data.batch(batch_size, drop_remainder=True)

## Result file

In [11]:
# header = "model_name", "fair_coeff", "clas_acc", "dp", "deqodds", "deqopp", "trade_dp", "trade_deqodds", "trade_deqopp", "TN_a0", "FP_a0", "FN_a0", "TP_a0", "TN_a1", "FP_a1", "FN_a1", "TP_a1"
# results = []

## Testing loop
#### Each model is evalueted 5 times
#### In the end of each iteration we save the result

### BEUTEL for DP

In [12]:
fairdef = 'DemPar'

for FAIR_COEFF in FAIR_COEFFS:
    for i in range(test_loop):

        opt = Adagrad(learning_rate=learning_rate)
    
        model = Beutel(xdim, ydim, adim, zdim, FAIR_COEFF, fairdef)

        ret = beutel_train(model, raw_data, train_data, epochs, opt)

        Y, A, Y_hat, A_hat = fair_evaluation(model, valid_data)
        clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(Y, A, Y_hat, A_hat, adim)

        fair_metrics = (dp, deqodds, deqopp)
        tradeoff = []
        for fair_metric in fair_metrics:
            tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

        result = ['BEUTEL4DP', FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

        # results.append(result)

> Epoch | Model Loss | Class Loss | Adv Loss | Class Acc | Adv Acc


2022-05-25 19:28:01.498623: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-05-25 19:28:01.684789: W tensorflow/python/util/util.cc:329] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


> 1 | 0.7122963666915894 | 0.6961708664894104 | 0.7284219264984131 | 0.7141080901856764 | 0.34511770557029176
> 2 | 0.7108821272850037 | 0.6953566074371338 | 0.7264076471328735 | 0.7446120689655172 | 0.3270474137931034
> 3 | 0.7100515961647034 | 0.694823145866394 | 0.7252800464630127 | 0.7475961538461539 | 0.3251409151193634
> 4 | 0.7095214128494263 | 0.694439709186554 | 0.7246031761169434 | 0.74809350132626 | 0.324809350132626
> 5 | 0.7091580629348755 | 0.6941453814506531 | 0.7241706848144531 | 0.7488809681697612 | 0.3246021220159151
> 6 | 0.7088955640792847 | 0.693909764289856 | 0.7238813042640686 | 0.7489638594164456 | 0.3245192307692308
> 7 | 0.7086985111236572 | 0.6937150955200195 | 0.7236818671226501 | 0.7489224137931034 | 0.32456067639257297
> 8 | 0.7085459232330322 | 0.6935502886772156 | 0.7235414981842041 | 0.7488395225464191 | 0.32439489389920423
> 9 | 0.7084248065948486 | 0.6934083700180054 | 0.7234411835670471 | 0.7487980769230769 | 0.32435344827586204
> 10 | 0.708326578140

### BEUTEL for Eq Opp

In [13]:
# fairdef = 'EqOpp'

# for FAIR_COEFF in FAIR_COEFFS:
#     for i in range(test_loop):

#         opt = Adam(learning_rate=learning_rate)

#         model = Beutel(xdim, ydim, adim, zdim, hidden_layer_specs, fairdef)

#         ret = beutel_train(model, raw_data, train_data, epochs, opt)

#         Y, A, Y_hat, A_hat = fair_evaluation(model, valid_data)
#         clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(Y, A, Y_hat, A_hat, adim)

#         fair_metrics = (dp, deqodds, deqopp)
#         tradeoff = []
#         for fair_metric in fair_metrics:
#             tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

#         result = ['BEUTEL4EqOpp', FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

#         # results.append(result)

## Saving into DF then CSV

In [14]:
# result_df = pd.DataFrame(results, columns=header)
# result_df

In [15]:
# result_df.to_csv(f'{data_name}-result/beutel-{epochs}.csv')